In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("aluminum_5_50nm step1.25nm.csv")
data.head()

,Unnamed: 0,R_nm,wavelength_nm,abs
0,0,5.0,300.000000,0.025922
1,1,5.0,301.820536,0.025442
2,2,5.0,303.652120,0.024977
3,3,5.0,305.494819,0.024522
4,4,5.0,307.348700,0.024075


In [3]:
radius = np.arange(0.5, 5.1, 0.125)
N = radius.shape[0]
N

37

In [4]:
features = ["wavelength_nm", "abs"]
a = data[features]
a = a.values
a = a.reshape(N, 200, 2)
a

array([[[3.00000000e+02, 2.59216383e-02],
        [3.01820536e+02, 2.54415458e-02],
        [3.03652120e+02, 2.49772809e-02],
        ...,
        [9.87972684e+02, 1.91190345e-03],
        [9.93968150e+02, 1.84624558e-03],
        [1.00000000e+03, 1.78264145e-03]],

       [[3.00000000e+02, 2.59216383e-02],
        [3.01820536e+02, 2.54415458e-02],
        [3.03652120e+02, 2.49772809e-02],
        ...,
        [9.87972684e+02, 1.91190345e-03],
        [9.93968150e+02, 1.84624558e-03],
        [1.00000000e+03, 1.78264145e-03]],

       [[3.00000000e+02, 1.09503944e-01],
        [3.01820536e+02, 1.06138676e-01],
        [3.03652120e+02, 1.02940522e-01],
        ...,
        [9.87972684e+02, 4.57338918e-03],
        [9.93968150e+02, 4.41329223e-03],
        [1.00000000e+03, 4.25834698e-03]],

       ...,

       [[3.00000000e+02, 9.86034858e-01],
        [3.01820536e+02, 9.90700324e-01],
        [3.03652120e+02, 9.96301199e-01],
        ...,
        [9.87972684e+02, 1.82818734e-01],
     

In [5]:
b = data["R_nm"]
b = b.values
b = b.reshape(N, 200, 1)
b

array([[[ 5.  ],
        [ 5.  ],
        [ 5.  ],
        ...,
        [ 5.  ],
        [ 5.  ],
        [ 5.  ]],

       [[ 6.25],
        [ 6.25],
        [ 6.25],
        ...,
        [ 6.25],
        [ 6.25],
        [ 6.25]],

       [[ 7.5 ],
        [ 7.5 ],
        [ 7.5 ],
        ...,
        [ 7.5 ],
        [ 7.5 ],
        [ 7.5 ]],

       ...,

       [[47.5 ],
        [47.5 ],
        [47.5 ],
        ...,
        [47.5 ],
        [47.5 ],
        [47.5 ]],

       [[48.75],
        [48.75],
        [48.75],
        ...,
        [48.75],
        [48.75],
        [48.75]],

       [[50.  ],
        [50.  ],
        [50.  ],
        ...,
        [50.  ],
        [50.  ],
        [50.  ]]])

### Spliting dataset

In [6]:
from sklearn.model_selection import train_test_split
a_train, a_valid, b_train, b_valid = train_test_split(a, b, train_size=0.85, test_size=0.15, random_state = 1)

In [7]:
a_train.shape

(31, 200, 2)

In [8]:
a_train = a_train.reshape(31, 200*2)
b_train = b_train.reshape(31, 200*1)
a_valid = a_valid.reshape(6, 200*2)
b_valid.shape
a_valid.shape

(6, 400)

In [9]:
a_valid

array([[3.00000000e+02, 1.09503944e-01, 3.01820536e+02, ...,
        4.41329223e-03, 1.00000000e+03, 4.25834698e-03],
       [3.00000000e+02, 1.26257042e+00, 3.01820536e+02, ...,
        1.69599632e-01, 1.00000000e+03, 1.69420569e-01],
       [3.00000000e+02, 1.73239842e-01, 3.01820536e+02, ...,
        5.24965781e-03, 1.00000000e+03, 5.06523327e-03],
       [3.00000000e+02, 1.16520227e+00, 3.01820536e+02, ...,
        1.83751647e-01, 1.00000000e+03, 1.87924835e-01],
       [3.00000000e+02, 1.21122644e+00, 3.01820536e+02, ...,
        1.55256693e-01, 1.00000000e+03, 1.53624649e-01],
       [3.00000000e+02, 1.26665779e+00, 3.01820536e+02, ...,
        1.64715870e-01, 1.00000000e+03, 1.63887378e-01]])

### 1.ExtraTreesRegressor

In [50]:
from sklearn.ensemble import ExtraTreesRegressor
et_model = ExtraTreesRegressor(random_state =1)
et_model.fit(a_train, b_train)

ExtraTreesRegressor(random_state=1)

In [51]:
et_b_preds = et_model.predict(a_valid)
et_b_preds = et_b_preds.reshape(6, 200, 1)
et_b_preds.shape

(6, 200, 1)

In [52]:
from sklearn.metrics import mean_squared_error, r2_score
print("MSR ExtraTreesRegressor: %f" %mean_squared_error(b_valid[:, 0, 0], et_b_preds[:, 0, 0]))
print("R2 ExtraTreesRegressor: %f %%" %(100*r2_score(b_valid[:, 0, 0], et_b_preds[:, 0, 0])))

MSR ExtraTreesRegressor: 1.308776
R2 ExtraTreesRegressor: 99.328115 %


In [54]:
pd.DataFrame({"valid": b_valid[:, 0 , 0] ,"et_preds": et_b_preds[:,0, 0]})

,valid,et_preds
0,7.50,8.9500
1,41.25,42.4750
2,8.75,10.0000
3,32.50,31.5375
4,36.25,37.2250
5,38.75,39.6500


### 2.RandomForestRegressor

In [55]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state =1)
rf_model.fit(a_train, b_train)

RandomForestRegressor(random_state=1)

In [56]:
rf_b_preds = rf_model.predict(a_valid)
rf_b_preds = rf_b_preds.reshape(6, 200, 1)
rf_b_preds

array([[[10.134375  ],
        [10.134375  ],
        [10.134375  ],
        ...,
        [10.134375  ],
        [10.134375  ],
        [10.134375  ]],

       [[42.6125    ],
        [42.6125    ],
        [42.6125    ],
        ...,
        [42.6125    ],
        [42.6125    ],
        [42.6125    ]],

       [[10.34895833],
        [10.34895833],
        [10.34895833],
        ...,
        [10.34895833],
        [10.34895833],
        [10.34895833]],

       [[31.475     ],
        [31.475     ],
        [31.475     ],
        ...,
        [31.475     ],
        [31.475     ],
        [31.475     ]],

       [[37.0125    ],
        [37.0125    ],
        [37.0125    ],
        ...,
        [37.0125    ],
        [37.0125    ],
        [37.0125    ]],

       [[39.125     ],
        [39.125     ],
        [39.125     ],
        ...,
        [39.125     ],
        [39.125     ],
        [39.125     ]]])

In [57]:
from sklearn.metrics import mean_squared_error, r2_score

print("MSE RandomForestRegressor: %f" % mean_squared_error(b_valid[:, 0, 0], rf_b_preds[:, 0, 0]))
print("R2 RandomForestRegressor: %f %%" %(100*r2_score(b_valid[:, 0, 0], rf_b_preds[:, 0, 0])))

MSE RandomForestRegressor: 2.187610
R2 RandomForestRegressor: 98.876949 %


In [58]:
pd.DataFrame({"valid": b_valid[:, 0 , 0] ,"preds": rf_b_preds[:,0, 0]})

,valid,preds
0,7.50,10.134375
1,41.25,42.612500
2,8.75,10.348958
3,32.50,31.475000
4,36.25,37.012500
5,38.75,39.125000


### 3.DecisionTreeRegressor

In [59]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 1)
dt_model.fit(a_train, b_train)

DecisionTreeRegressor(random_state=1)

In [60]:
b_preds = dt_model.predict(a_valid)
b_preds = b_preds.reshape(6, 200, 1)
b_preds

array([[[10. ],
        [10. ],
        [10. ],
        ...,
        [10. ],
        [10. ],
        [10. ]],

       [[42.5],
        [42.5],
        [42.5],
        ...,
        [42.5],
        [42.5],
        [42.5]],

       [[10. ],
        [10. ],
        [10. ],
        ...,
        [10. ],
        [10. ],
        [10. ]],

       [[37.5],
        [37.5],
        [37.5],
        ...,
        [37.5],
        [37.5],
        [37.5]],

       [[35. ],
        [35. ],
        [35. ],
        ...,
        [35. ],
        [35. ],
        [35. ]],

       [[40. ],
        [40. ],
        [40. ],
        ...,
        [40. ],
        [40. ],
        [40. ]]])

In [61]:
from sklearn.metrics import mean_squared_error, r2_score
print("MSE DecisionTreeRegressor: %f" % mean_squared_error(b_valid[:, 0, 0], b_preds[:, 0, 0]))
print("R2 DecisionTreeRegressor: %f %%" %(100*r2_score(b_valid[:, 0, 0], b_preds[:, 0, 0])))

MSE DecisionTreeRegressor: 6.250000
R2 DecisionTreeRegressor: 96.791444 %


In [63]:
pd.DataFrame({"valid": b_valid[:, 0 , 0] ,"preds": b_preds[:,0, 0]})

,valid,preds
0,7.50,10.0
1,41.25,42.5
2,8.75,10.0
3,32.50,37.5
4,36.25,35.0
5,38.75,40.0


### 4.KNeighborRegressor

In [64]:
from sklearn.neighbors import KNeighborsRegressor
n_neighbors = 2
for i in enumerate(["uniform"]):
    kn_model = KNeighborsRegressor(n_neighbors)
    kn_model.fit(a_train, b_train)

In [65]:
kn_b_preds = kn_model.predict(a_valid)
kn_b_preds = kn_b_preds.reshape(6, 200, 1)
kn_b_preds

array([[[ 8.125],
        [ 8.125],
        [ 8.125],
        ...,
        [ 8.125],
        [ 8.125],
        [ 8.125]],

       [[43.125],
        [43.125],
        [43.125],
        ...,
        [43.125],
        [43.125],
        [43.125]],

       [[10.625],
        [10.625],
        [10.625],
        ...,
        [10.625],
        [10.625],
        [10.625]],

       [[30.625],
        [30.625],
        [30.625],
        ...,
        [30.625],
        [30.625],
        [30.625]],

       [[36.25 ],
        [36.25 ],
        [36.25 ],
        ...,
        [36.25 ],
        [36.25 ],
        [36.25 ]],

       [[38.75 ],
        [38.75 ],
        [38.75 ],
        ...,
        [38.75 ],
        [38.75 ],
        [38.75 ]]])

In [66]:
from sklearn.metrics import mean_squared_error, r2_score
print("MSE KNeighborsRegressor: %f" % mean_squared_error(b_valid[:, 0, 0], kn_b_preds[:, 0, 0]))
print("R2 KNeighborsRegressor: %f %%" %(100* r2_score(b_valid[:, 0, 0], kn_b_preds[:, 0, 0])))

MSE KNeighborsRegressor: 1.822917
R2 KNeighborsRegressor: 99.064171 %


In [67]:
pd.DataFrame({"valid": b_valid[:, 0 , 0] ,"preds": kn_b_preds[:,0, 0]})

,valid,preds
0,7.50,8.125
1,41.25,43.125
2,8.75,10.625
3,32.50,30.625
4,36.25,36.250
5,38.75,38.750


### 5.Linear Regressor

In [68]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(a_train, b_train)

LinearRegression()

In [69]:
linear_preds = linear.predict(a_valid)
linear_preds = linear_preds.reshape(6, 200, 1)
linear_preds

array([[[ 8.57213014],
        [ 8.57213014],
        [ 8.57213014],
        ...,
        [ 8.57213014],
        [ 8.57213014],
        [ 8.57213014]],

       [[41.39796501],
        [41.39796501],
        [41.39796501],
        ...,
        [41.39796501],
        [41.39796501],
        [41.39796501]],

       [[10.        ],
        [10.        ],
        [10.        ],
        ...,
        [10.        ],
        [10.        ],
        [10.        ]],

       [[26.68624003],
        [26.68624003],
        [26.68624003],
        ...,
        [26.68624003],
        [26.68624003],
        [26.68624003]],

       [[37.39667327],
        [37.39667327],
        [37.39667327],
        ...,
        [37.39667327],
        [37.39667327],
        [37.39667327]],

       [[37.41576625],
        [37.41576625],
        [37.41576625],
        ...,
        [37.41576625],
        [37.41576625],
        [37.41576625]]])

In [70]:
from sklearn.metrics import mean_squared_error, r2_score
print("MSE LinearRegressor: %f" % mean_squared_error(b_valid[:, 0, 0], linear_preds[:, 0, 0]))
print("R2: %f %%" % (100*r2_score(b_valid[:, 0, 0], linear_preds[:, 0, 0])))

MSE LinearRegressor: 6.604783
R2: 96.609309 %


In [71]:
pd.DataFrame({"valid": b_valid[:, 0 , 0] ,"preds": linear_preds[:,0, 0]})

,valid,preds
0,7.50,8.572130
1,41.25,41.397965
2,8.75,10.000000
3,32.50,26.686240
4,36.25,37.396673
5,38.75,37.415766


In [73]:
data1 = pd.read_csv("data_test_aluminum_26nm_200wavelengths.csv")
data1

,Unnamed: 0,wavelength_nm,abs
0,0,300.000000,0.801921
1,1,301.820536,0.782630
2,2,303.652120,0.765067
3,3,305.494819,0.749244
4,4,307.348700,0.735122
...,...,...,...
195,195,976.090025,0.058272
196,196,982.013382,0.055146
197,197,987.972684,0.051998
198,198,993.968150,0.048963


In [74]:
data1 = data1.values

In [75]:
collumn = data1.shape[1]
row = data1.shape[0]
print(collumn, row)

3 200


In [76]:
for j in np.arange(0, collumn, 1):
    for i in np.arange(1, row, 1):
        if(data1[i, j] == data1[i-1, j]+1):
            data1 = np.delete(data1, j, 1)
    break

In [77]:
data1 = data1.reshape(1, row*2)
data1.shape

(1, 400)

In [78]:
linear = linear.predict(data1)
r = linear[0, 0]
r

25.228549684783502

### Save model ExtraTressRegressor


In [77]:
import joblib

joblib.dump(et_model, 'ExtraRegressor preds R.joblib')

['ExtraRegressor preds R.joblib']